# Use Custom Template Notebook

In [61]:
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#Modeling Libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier

from xgboost import XGBClassifier
#from catboost import CatBoostClassifier

from joblib import dump, load
import requests

# Initialize Snowflake Session using Default connection

In [62]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [63]:
table_name = 'AUTO_INSURANCE_CLAIMS_DATA'

In [64]:
sf_df = my_session.sql("select * from {}".format(table_name))

In [65]:
df = sf_df.to_pandas()

In [66]:
df.head()

,MONTHS_AS_CUSTOMER,CUSTOMER_AGE,POLICY_NUMBER,POLICY_BIND_DATE,POLICY_STATE,POLICY_CSL,POLICY_DEDUCTABLE,POLICY_ANNUAL_PREMIUM,UMBRELLA_LIMIT,INSURED_ZIP,...,POLICE_REPORT_AVAILABLE,TOTAL_CLAIM_AMOUNT_PAID,INJURY_CLAIM,PROPERTY_CLAIM,VEHICLE_CLAIM,AUTO_MAKE,AUTO_MODEL,AUTO_YEAR,FRAUD_REPORTED,POLICY_BIND_DATE_CUSTOM
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005,No Fraud Reported,2011-08-17
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,Police Report Available,64579,11170,5093,48316,Audi,A3,1999,No Fraud Reported,2010-09-27
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014,No Fraud Reported,2002-01-13
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005,No Fraud Reported,2012-02-07
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,Police Report Available,94511,412,14545,79554,Ford,Escape,2013,No Fraud Reported,2008-06-18


In [67]:
#insurance_claim =  df.copy()
insurance_claim =  df[::1]

In [68]:
insurance_claim = insurance_claim[['MONTHS_AS_CUSTOMER', 'CUSTOMER_AGE', 'POLICY_NUMBER',
       'POLICY_BIND_DATE', 'POLICY_STATE', 'POLICY_CSL', 'POLICY_DEDUCTABLE',
       'POLICY_ANNUAL_PREMIUM', 'UMBRELLA_LIMIT', 'INSURED_ZIP', 'INSURED_SEX',
       'INSURED_EDUCATION_LEVEL', 'INSURED_OCCUPATION', 'INSURED_HOBBIES',
       'INSURED_RELATIONSHIP', 'CAPITAL_GAINS', 'CAPITAL_LOSS',
       'INCIDENT_DATE', 'INCIDENT_TYPE', 'COLLISION_TYPE', 'INCIDENT_SEVERITY',
       'AUTHORITIES_CONTACTED', 'INCIDENT_STATE', 'INCIDENT_CITY',
       'INCIDENT_LOCATION', 'INCIDENT_HOUR_OF_THE_DAY', 'INCIDENT_TIME_OF_DAY',
       'NUMBER_OF_VEHICLES_INVOLVED', 'PROPERTY_DAMAGE', 'BODILY_INJURIES',
       'WITNESSES', 'POLICE_REPORT_AVAILABLE', 'TOTAL_CLAIM_AMOUNT_PAID',
       'INJURY_CLAIM', 'PROPERTY_CLAIM', 'VEHICLE_CLAIM', 'AUTO_MAKE',
       'AUTO_MODEL', 'AUTO_YEAR', 'FRAUD_REPORTED']]

In [69]:
insurance_claim.columns = insurance_claim.columns.str.lower()

In [70]:
insurance_claim.columns

Index(['months_as_customer', 'customer_age', 'policy_number',
       'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'capital_gains', 'capital_loss',
       'incident_date', 'incident_type', 'collision_type', 'incident_severity',
       'authorities_contacted', 'incident_state', 'incident_city',
       'incident_location', 'incident_hour_of_the_day', 'incident_time_of_day',
       'number_of_vehicles_involved', 'property_damage', 'bodily_injuries',
       'witnesses', 'police_report_available', 'total_claim_amount_paid',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_make',
       'auto_model', 'auto_year', 'fraud_reported'],
      dtype='object')

In [71]:
insurance_claim.rename(columns = {'total_claim_amount_paid': 'total_claim_amount'}, inplace=True)

In [72]:
insurance_claim=insurance_claim.replace("?",np.NaN)

In [73]:
df = insurance_claim[::1]

In [74]:
#df_copy = insurance_claim.copy()

In [75]:
#actual_inference = df.copy()
actual_inference = df[::1]
actual_inference.drop('fraud_reported', axis = 1, inplace=True)

In [76]:
actual_inference.head()

,months_as_customer,customer_age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year
0,139,37,457069,17-08-2011,CT,250/500,1351,1589,16139812,609322,...,0,3,No Police Report Available,48182,538,20189,27455,Dodge,Neon,2005
1,292,52,429086,27-09-2010,CT,100/300,2216,2669,15406514,438830,...,1,2,Police Report Available,64579,11170,5093,48316,Audi,A3,1999
2,45,20,457562,13-01-2002,CT,100/300,929,1290,5629268,616164,...,0,2,No Police Report Available,32420,538,845,31037,Mercedes,E400,2014
3,49,67,471692,07-02-2012,CT,500/1000,699,970,18501194,445120,...,2,1,No Police Report Available,53780,7280,898,45602,Suburu,Legacy,2005
4,128,45,463009,18-06-2008,CT,100/300,1122,1384,16890664,442936,...,0,1,Police Report Available,94511,412,14545,79554,Ford,Escape,2013


In [77]:
# dropping columns which are not necessary for prediction

to_drop = ['policy_number','policy_bind_date','policy_state','insured_zip','incident_location','incident_date',
           'incident_state','incident_city','insured_hobbies','auto_make','auto_model','auto_year']

df.drop(to_drop, inplace=True, axis=1)

In [78]:
df['incident_hour_of_the_day'] = df['incident_hour_of_the_day'].astype(str).astype(int)

In [79]:
df['incident_hour_of_the_day'].dtype

dtype('int64')

In [80]:
# separating the feature and target columns

X = df.drop('fraud_reported', axis = 1)
y = df['fraud_reported']

In [81]:
# extracting categorical columns
cat_df = X.select_dtypes(include = ['object'])

In [82]:
cat_df.columns

Index(['policy_csl', 'insured_sex', 'insured_education_level',
       'insured_occupation', 'insured_relationship', 'incident_type',
       'collision_type', 'incident_severity', 'authorities_contacted',
       'incident_time_of_day', 'property_damage', 'police_report_available'],
      dtype='object')

In [83]:
# printing unique values of each column
for col in cat_df.columns:
    print(f"{col}: \n{cat_df[col].unique()}\n")

policy_csl: 
['250/500' '100/300' '500/1000']

insured_sex: 
['MALE' 'FEMALE']

insured_education_level: 
['High School' 'MD' 'Masters' 'Associate' 'College' 'PhD' 'JD']

insured_occupation: 
['handlers-cleaners' 'armed-forces' 'prof-specialty' 'sales'
 'machine-op-inspct' 'priv-house-serv' 'adm-clerical' 'exec-managerial'
 'tech-support' 'farming-fishing' 'craft-repair' 'transport-moving'
 'protective-serv' 'other-service']

insured_relationship: 
['wife' 'unmarried' 'other-relative' 'husband' 'own-child' 'not-in-family']

incident_type: 
['Single Vehicle Collision' 'Parked Car' 'Multi-vehicle Collision'
 'Vehicle Theft']

collision_type: 
['Front Collision' 'Rear Collision' 'Side Collision'
 'Details not Available']

incident_severity: 
['Minor Damage' 'Major Damage' 'Total Loss']

authorities_contacted: 
['Ambulance' 'Other' 'None' 'Police' 'Fire']

incident_time_of_day: 
['Morning to Noon' 'Night Time' 'Afternoon Hours' 'Early Morning Hours']

property_damage: 
['Property Damage' '

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232025 entries, 0 to 232024
Data columns (total 28 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   months_as_customer           232025 non-null  int16 
 1   customer_age                 232025 non-null  int8  
 2   policy_csl                   232025 non-null  object
 3   policy_deductable            232025 non-null  int16 
 4   policy_annual_premium        232025 non-null  int16 
 5   umbrella_limit               232025 non-null  int32 
 6   insured_sex                  232025 non-null  object
 7   insured_education_level      232025 non-null  object
 8   insured_occupation           232025 non-null  object
 9   insured_relationship         232025 non-null  object
 10  capital_gains                232025 non-null  int32 
 11  capital_loss                 232025 non-null  int32 
 12  incident_type                232025 non-null  object
 13  collision_type

In [85]:
cat_df = pd.get_dummies(cat_df, drop_first = True)

In [86]:
num_df = df.select_dtypes(include = ['int16','int8','int32','float64','int64'])

In [87]:
num_df.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim'],
      dtype='object')

In [88]:
cat_df.columns

Index(['policy_csl_250/500', 'policy_csl_500/1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed-forces',
       'insured_occupation_craft-repair', 'insured_occupation_exec-managerial',
       'insured_occupation_farming-fishing',
       'insured_occupation_handlers-cleaners',
       'insured_occupation_machine-op-inspct',
       'insured_occupation_other-service',
       'insured_occupation_priv-house-serv',
       'insured_occupation_prof-specialty',
       'insured_occupation_protective-serv', 'insured_occupation_sales',
       'insured_occupation_tech-support',
       'insured_occupation_transport-moving',
       'insured_relationship_not-in-family',
       'insured_relationship_other-relative', 'insured_relationship_own-child',
       'insured_relationship

In [89]:
# combining the Numerical and Categorical dataframes to get the final dataset
X = pd.concat([num_df, cat_df], axis = 1)

In [90]:
X.columns = X.columns.str.replace('/', '_')
X.columns = X.columns.str.replace(' ', '_')
X.columns = X.columns.str.replace('-', '_')

In [91]:
# splitting data into training set and test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [92]:
num_df = X_train[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [93]:
num_df_test = X_test[['months_as_customer', 'policy_deductable', 'umbrella_limit',
       'capital_gains', 'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'injury_claim', 'property_claim',
       'vehicle_claim']]

In [94]:
# Scaling the numeric values in the dataset
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(num_df)

In [95]:
scaled_data_test = scaler.transform(num_df_test)

In [96]:
scaled_num_df = pd.DataFrame(data = scaled_data, columns = num_df.columns, index = X_train.index)
scaled_num_df.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
39980,0.069084,-1.049089,-1.759318,-0.534261,0.381451,0.163846,2.176905,1.226680,1.341097,-0.633652,-0.969286,0.297673
159740,-0.908273,-1.353772,-0.671723,-0.774332,1.727854,-1.029732,-0.677552,-1.218461,-0.451056,-1.034111,0.177071,1.529212
45156,-0.996126,-0.516363,-0.181555,-0.679271,0.203333,-0.688710,0.749677,0.004110,-1.347132,0.500189,-0.947666,1.561866
98486,0.299697,0.870595,1.602866,-0.798368,-0.685869,-1.882288,0.749677,0.004110,1.341097,-0.801442,-0.953262,0.370380
9198,1.760243,-0.226635,0.037148,-0.288777,-0.128970,1.186913,0.749677,0.004110,-0.451056,1.559223,-0.983656,-0.943049


In [97]:
scaled_num_df_test = pd.DataFrame(data = scaled_data_test, columns = num_df_test.columns, index = X_test.index)
scaled_num_df_test.head()

,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,property_claim,vehicle_claim
151850,2.089689,1.779034,0.544536,-1.603067,-1.148463,0.504869,0.749677,-1.218461,1.341097,-1.038026,0.901969,-0.411593
7368,-1.281646,1.141631,0.636505,-1.361841,0.174594,-0.859221,-0.677552,0.004110,-1.347132,-0.388539,0.126964,0.967334
195172,-0.205454,0.326653,1.591521,-1.426731,-0.287831,1.016402,-0.677552,0.004110,-0.451056,0.678605,-0.457024,0.143715
228601,0.914664,-1.164981,-0.778254,1.007215,1.609867,1.186913,3.604133,-1.218461,1.341097,-1.015514,0.616079,1.827179
221197,-0.315270,0.220108,0.998459,0.501702,-1.294346,-1.200243,-0.677552,1.226680,-1.347132,1.832720,0.907183,0.016862


In [98]:
X_train.drop(columns = scaled_num_df.columns, inplace = True)

In [99]:
X_train = pd.concat([scaled_num_df, X_train], axis = 1)

In [100]:
X_test.drop(columns = scaled_num_df_test.columns, inplace = True)

In [101]:
X_test = pd.concat([scaled_num_df_test, X_test], axis = 1)

# Decision Tree

In [102]:
#from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

DecisionTreeClassifier()

In [103]:
y_pred = dtc.predict(X_test)

In [104]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9931042805178686
[[ 8539   170]
 [  230 49068]]
                   precision    recall  f1-score   support

   Fraud Reported       0.97      0.98      0.98      8709
No Fraud Reported       1.00      1.00      1.00     49298

         accuracy                           0.99     58007
        macro avg       0.99      0.99      0.99     58007
     weighted avg       0.99      0.99      0.99     58007



In [105]:
from fosforml import register_model

In [106]:
X.columns

Index(['months_as_customer', 'customer_age', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'capital_gains',
       'capital_loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim',
       'policy_csl_250_500', 'policy_csl_500_1000', 'insured_sex_MALE',
       'insured_education_level_College',
       'insured_education_level_High_School', 'insured_education_level_JD',
       'insured_education_level_MD', 'insured_education_level_Masters',
       'insured_education_level_PhD', 'insured_occupation_armed_forces',
       'insured_occupation_craft_repair', 'insured_occupation_exec_managerial',
       'insured_occupation_farming_fishing',
       'insured_occupation_handlers_cleaners',
       'insured_occupation_machine_op_inspct',
       'insured_occupation_other_service',
       'insured_occupation_priv_house_serv',
       'insured_occupati

In [107]:
y_pred_series = pd.Series(y_pred)

In [169]:
type(X_train), type(X_test),type(y_train),type(y_test), type(y_pred_series), 

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [153]:
#X_train_df = pd.DataFrame(X_train)
#y_train_df = pd.DataFrame(y_train)
#y_train_df = pd.DataFrame({'target':y_train.index, 'name':y_train.values})

#X_test_df = pd.DataFrame(X_test)
#y_test_df = pd.DataFrame(y_test)
#y_test_df = pd.DataFrame({'target':y_test.index, 'name':y_test.values})

#y_pred_df = pd.DataFrame(y_pred)
#y_pred_df = pd.DataFrame({'target':y_pred_series.index, 'name':y_pred_series.values})

In [155]:
type(X_train_df), type(y_train_df), type(X_test_df), type(y_test_df), type(y_pred_df)

(pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame,
 pandas.core.frame.DataFrame)

In [170]:
type(X_train), type(y_train), type(X_test), type(y_test), type(y_pred_series)

(pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.frame.DataFrame,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [151]:
#X_train_df.name = 'X_train_df' 
#y_train_df.name = 'y_train_df' 
#X_test_df.name = 'X_test_df' 
#y_test_df.name = 'y_test_df' 
#y_pred_df.name = 'y_pred_df'

In [152]:
#X_train_df.columns.name = 'X_train_df' 
#y_train_df.columns.name = 'y_train_df' 
#X_test_df.columns.name = 'X_test_df' 
#y_test_df.columns.name = 'y_test_df' 
#y_pred_df.columns.name = 'y_pred_df'

In [171]:
y_train.squeeze().name

'fraud_reported'

In [172]:
y_test.squeeze().name

'fraud_reported'

In [192]:
y_pred_series.name = 'fraud_reported'

In [194]:
y_pred_series

0        No Fraud Reported
1        No Fraud Reported
2        No Fraud Reported
3        No Fraud Reported
4        No Fraud Reported
               ...        
58002    No Fraud Reported
58003    No Fraud Reported
58004    No Fraud Reported
58005    No Fraud Reported
58006    No Fraud Reported
Name: fraud_reported, Length: 58007, dtype: object

In [186]:
y_test

151850    No Fraud Reported
7368      No Fraud Reported
195172    No Fraud Reported
228601    No Fraud Reported
221197    No Fraud Reported
                ...        
43097     No Fraud Reported
208782    No Fraud Reported
31954     No Fraud Reported
155988    No Fraud Reported
191091    No Fraud Reported
Name: fraud_reported, Length: 58007, dtype: object

In [196]:
y_pred_series.values

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [197]:
y_test.values

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [198]:
y_train.values

array(['No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported', ...,
       'No Fraud Reported', 'No Fraud Reported', 'No Fraud Reported'],
      dtype=object)

In [203]:
y_pred_series.name, y_train.name, y_test.name

('Panoti', 'fraud_reported', 'fraud_reported')

In [202]:
y_pred_series.name = 'Panoti'

In [214]:
import re
def get_clean_col(col):
    '''
    Get the column name without quotes
    '''
    match_string = re.match(r"(['\"]+)(.*)\1", col)
    if match_string:
        return match_string.group(2)
    return col


In [216]:
[(col, get_clean_col(col)) for col in X_test.columns.to_list()]

[('months_as_customer', 'months_as_customer'),
 ('policy_deductable', 'policy_deductable'),
 ('umbrella_limit', 'umbrella_limit'),
 ('capital_gains', 'capital_gains'),
 ('capital_loss', 'capital_loss'),
 ('incident_hour_of_the_day', 'incident_hour_of_the_day'),
 ('number_of_vehicles_involved', 'number_of_vehicles_involved'),
 ('bodily_injuries', 'bodily_injuries'),
 ('witnesses', 'witnesses'),
 ('injury_claim', 'injury_claim'),
 ('property_claim', 'property_claim'),
 ('vehicle_claim', 'vehicle_claim'),
 ('customer_age', 'customer_age'),
 ('policy_annual_premium', 'policy_annual_premium'),
 ('total_claim_amount', 'total_claim_amount'),
 ('policy_csl_250_500', 'policy_csl_250_500'),
 ('policy_csl_500_1000', 'policy_csl_500_1000'),
 ('insured_sex_MALE', 'insured_sex_MALE'),
 ('insured_education_level_College', 'insured_education_level_College'),
 ('insured_education_level_High_School',
  'insured_education_level_High_School'),
 ('insured_education_level_JD', 'insured_education_level_JD'),

In [222]:
from sklearn import preprocessing 
# label_encoder object knows  
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
y_test_lb= label_encoder.fit_transform(y_test) 

In [224]:
y_test_lb, type(y_test_lb)

(array([1, 1, 1, ..., 1, 1, 1]), numpy.ndarray)

In [225]:
y_train_lb= label_encoder.transform(y_train) 
y_pred_lb= label_encoder.transform(y_pred) 

In [226]:
y_train_lb, type(y_train_lb)

(array([1, 1, 1, ..., 1, 1, 1]), numpy.ndarray)

In [227]:
y_pred_lb, type(y_pred_lb)

(array([1, 1, 1, ..., 1, 1, 1]), numpy.ndarray)

In [228]:
y_train_sr = pd.Series(y_train_lb)
y_test_sr = pd.Series(y_test_lb)
y_pred_sr = pd.Series(y_pred_lb)

In [231]:
type(y_train_sr), type(y_test_sr), type(y_pred_sr)

(pandas.core.series.Series,
 pandas.core.series.Series,
 pandas.core.series.Series)

In [232]:
type(X_train), type(X_test)

(pandas.core.frame.DataFrame, pandas.core.frame.DataFrame)

In [235]:
y_pred_sr.name = 'fraud_classified'
y_train_sr.name = 'fraud_reported'
y_test_sr.name = 'fraud_reported'

In [240]:
X_train_temp=X_train.head(10000)
X_test_temp=X_test.head(10000)

In [242]:
y_train_temp = y_train_sr[:10000]
y_test_temp = y_test_sr[:10000]
y_pred_temp = y_pred_sr[:10000]

In [245]:
from snowflake.ml.registry import Registry
 
model_registry = Registry(session=my_session,
                          database_name='FDC_Insurance',
                          schema_name='INS_CLAIMS_RESERV_SCHEMA'
                         )

In [263]:
model_registry.log_model(dtc,
                         model_name="Claims_Fraud_DTree_Classifier",
                         version_name="v1",
                         comment="Insurance claims model",
                         conda_dependencies=["scikit-learn==1.3.2"],
                         metrics=[],
                         sample_input_data= X_test_temp.head(),
                         python_version='3.9')

NotImplementedError: (2103) Type object is not supported as a DataType.

In [261]:
object_columns = X_test_temp.select_dtypes(include='object').columns
print(object_columns)

Index([], dtype='object', name='X_test_df')


In [260]:
X_test_temp.head()

X_test_df,months_as_customer,policy_deductable,umbrella_limit,capital_gains,capital_loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,injury_claim,...,authorities_contacted_None,authorities_contacted_Other,authorities_contacted_Police,incident_time_of_day_Early_Morning_Hours,incident_time_of_day_Morning_to_Noon,incident_time_of_day_Night_Time,property_damage_No_Property_Damage,property_damage_Property_Damage,police_report_available_Police_Report_Available,police_report_available_Unknown
151850,2.089689,1.779034,0.544536,-1.603067,-1.148463,0.504869,0.749677,-1.218461,1.341097,-1.038026,...,False,False,True,False,False,True,False,True,True,False
7368,-1.281646,1.141631,0.636505,-1.361841,0.174594,-0.859221,-0.677552,0.004110,-1.347132,-0.388539,...,False,False,False,False,True,False,False,False,True,False
195172,-0.205454,0.326653,1.591521,-1.426731,-0.287831,1.016402,-0.677552,0.004110,-0.451056,0.678605,...,False,False,False,False,False,True,False,True,True,False
228601,0.914664,-1.164981,-0.778254,1.007215,1.609867,1.186913,3.604133,-1.218461,1.341097,-1.015514,...,False,False,True,False,False,True,False,True,False,True
221197,-0.315270,0.220108,0.998459,0.501702,-1.294346,-1.200243,-0.677552,1.226680,-1.347132,1.832720,...,False,False,False,False,True,False,False,True,False,True


In [257]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58007 entries, 151850 to 191091
Data columns (total 61 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   months_as_customer                               58007 non-null  float64
 1   policy_deductable                                58007 non-null  float64
 2   umbrella_limit                                   58007 non-null  float64
 3   capital_gains                                    58007 non-null  float64
 4   capital_loss                                     58007 non-null  float64
 5   incident_hour_of_the_day                         58007 non-null  float64
 6   number_of_vehicles_involved                      58007 non-null  float64
 7   bodily_injuries                                  58007 non-null  float64
 8   witnesses                                        58007 non-null  float64
 9   injury_claim               

In [243]:
## registering the model in Fosfor Insight Designer.
register_model(
    model_obj=dtc, 
    session=my_session,
    x_train=X_train_temp,
    y_train=y_train_temp,
    x_test=X_test_temp,
    y_test=y_test_temp,
    y_pred=y_pred_temp,
    source="Notebook",
    dataset_name="AUTO_INSURANCE_CLAIMS_DATA",
    dataset_source="Snowflake",
    #dataset_source="InMemory",
    name="Claims_Fraud_DTree_Classifier",
    description="Insurance claims model trained for Fraud Classification",
    flavour="sklearn",
    model_type="classification",
    conda_dependencies=["scikit-learn==1.3.2"]
)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceeding without creating optional arguments


Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might no

Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <function roc_curve.<locals>.roc_curve_anon_sproc at 0x7fcf018f2040>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.*'. Your UDF might not work when the package version is different between the server and your local enviro

Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Failed to register model 'MODEL_4CAC9B7E_5794_4BA1_8E11_719ABA32ECA5_FDC_CLAIMS_FRAUD_DTREE_CLASSIFIER'. (2103) Type object is not supported as a DataType."

In [237]:
type(dtc)

sklearn.tree._classes.DecisionTreeClassifier

In [238]:
X_train.shape

(174018, 61)

In [239]:
X_test.shape

(58007, 61)

# Version 2 of Decision Tree Fraud Classifier Model

In [125]:
dtc = DecisionTreeClassifier(random_state=7)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(random_state=7)

In [126]:
y_pred = dtc.predict(X_test)
y_prob = dtc.predict_proba(X_test)[:,1]

In [127]:
dtc_train_acc = accuracy_score(y_train, dtc.predict(X_train))
dtc_test_acc = accuracy_score(y_test, y_pred)

print(f"Training accuracy of Decision Tree is : {dtc_train_acc}")
print(f"Test accuracy of Decision Tree is : {dtc_test_acc}")

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Training accuracy of Decision Tree is : 1.0
Test accuracy of Decision Tree is : 0.9941263851657076
[[ 8305   156]
 [  188 49918]]
                   precision    recall  f1-score   support

   Fraud Reported       0.98      0.98      0.98      8461
No Fraud Reported       1.00      1.00      1.00     50106

         accuracy                           0.99     58567
        macro avg       0.99      0.99      0.99     58567
     weighted avg       0.99      0.99      0.99     58567



In [128]:
## registering the model in Fosfor Insight Designer using same name.
model_reg = register_model(dtc, 
               score, 
               name="Claims_Fraud_DTree_Classifier", 
               description="Insurance claim's model trained for Fraud Classification",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="snowflake-connector-python[pandas]",
               y_true=y_test,
               y_pred=y_pred, 
               prob=y_prob,
               features=X_train.columns,
               input_type="json", 
               explain_ai=True,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
